# Differential Privacy for PRETSA

In [1]:
!git clone https://github.com/cmalvr/PRETSA

Cloning into 'PRETSA'...
remote: Enumerating objects: 387, done.
remote: Counting objects: 100% (147/147), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 387 (delta 84), reused 95 (delta 38), pack-reused 240 (from 1)
Receiving objects: 100% (387/387), 11.48 MiB | 53.21 MiB/s, done.
Resolving deltas: 100% (222/222), done.


In [2]:
!pip install -r /content/PRETSA/requirements.txt

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.3 MB/s eta 0:00:00
  Created wheel for anytree: filename=anytree-2.4.3-py2.py3-none-any.whl size=26707 sha256=620cffed294d1f52bede1aa15beb65f4f42476c35502f56230845e7a2c263393
  Stored in directory: /root/.cache/pip/wheels/f3/d3/8f/9eea88a7fe27ca18e7ebfc23e1012514c8b13a4552aa43be12
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=3a5ea8c82eeb99050ef8f74b412dd0cbc52dd841717307bac3493d1674645dcd
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built anytree intervaltree


In [3]:
from pathlib import Path
import zipfile

# Define paths
zip_path = Path("/content/PRETSA/baselogs.zip")
extract_dir = Path("/content/PRETSA")

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Search for CSV files
csv_files = list(extract_dir.rglob("*.csv"))

if csv_files:
    print("Found CSV files:", [f.name for f in csv_files])
else:
    print("No CSV files found.")

Found CSV files: ['bpic2013_dataset.csv', 'Sepsis_dataset.csv', 'traffic_fines_dataset.csv', 'HospitalBilling_dataset.csv', 'CoSeLoG_dataset.csv']


In [3]:
!python /content/PRETSA/dataset_collection.py

/usr/local/lib/python3.11/dist-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/usr/local/lib/python3.11/dist-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 314, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:


Not patching Numba
  warnings.warn(msg, UserWarning)
/usr/local/lib/python3.11/dist-packages/cudf/utils/gpu_utils.py:62: UserWarning: Failed to dlopen libcuda.so.1
  warnings.warn(str(e))
Downloaded: /content/PRETSA/downloads/CoSeLoG.xes.gz
Processing CoSeLoG...
parsing log, completed traces :: 100% 1434/1434 [00:00<00:00, 2831.69it/s]
Save

## Running Annotations for three datasets

In [4]:
# Generates all the original annotation duration for each of the three datasets: CoSeLog, Sepsis and Road Traffc
# Generates /content/PRETSA/original_annotation/
# Seems to be the same as what is contained in baselogs but we will run in for sanity
!python /content/PRETSA/gc_add_annotations_duration.py with

Processing: CoSeLoG
Writting: /content/PRETSA/original_annotation/CoSeLoG_duration.csv
Reading: /content/PRETSA/csv_outputs/CoSeLoG_dataset.csv
Processing: Sepsis
Writting: /content/PRETSA/original_annotation/Sepsis_duration.csv
Reading: /content/PRETSA/csv_outputs/Sepsis_dataset.csv
Processing: Road_Traffic_Fine_Management_Process
Writting: /content/PRETSA/original_annotation/Road_Traffic_Fine_Management_Process_duration.csv
Reading: /content/PRETSA/csv_outputs/Road_Traffic_Fine_Management_Process_dataset.csv


## Run Pretsa: Generates Pretsa Log


In [9]:
%cd /content/PRETSA
!git pull

/content/PRETSA
Already up to date.


In [ ]:
!python /content/PRETSA/startExperimentsForJournalExtension_PRETSA.py
# Generates: /content/PRETSA/pretsalog/ f"{dataset}_t{t}_k{k}_pretsa.csv"
# These contain the new annotations for duration after Running PRETSA method

 Load Event Log: /content/PRETSA/original_annotation/Sepsis_duration.csv
Generated Distance Matrix
 Modified 760 cases for k=4, t=1.0
 Private event log saved at: /content/PRETSA/pretsalog/Sepsis_t1.0_k4_pretsa.csv
 Experiment metadata saved at: /content/PRETSA/pretsalog/Sepsis_t1.0_k4_pretsa.pickle
 Load Event Log: /content/PRETSA/original_annotation/Sepsis_duration.csv


## Run Baseline: Generate Baseline log

In [ ]:
!python /content/PRETSA/generate_baseline_log.py
# Generates: /content/PRETSA/baselinelogs/ Path(filePath).name.replace(".csv","_pretsa_baseline_k%s_t%s.csv" % (kString,tString))
# These contain the new annotations for duration after Running BASELINE method

## Calculate Statistics

In [8]:
!python /content/PRETSA/calculateBaselineEventLogStatistics.py #Generates: "baseline_event_logs_statistics/"
!python /content/PRETSA/calculatePRETSAEventLogStatistics.py  #Generates: "PRETSA_event_logs_statistics/"

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/calculateBaselineEventLogStatistics.py <dictName> #Needs: dictPath + dataset + "_duration_baseline_k" + str(k) + ".csv", generates: dictPath + "baseline_event_logs_statistics.csv"'
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/calculatePRETSAEventLogStatistics.py <dictName> #Needs: dictPath + dataset + "_duration_pretsa_k" + str(k) + ".csv", generates: dictPath + "pretsa_event_logs_statistics.csv"'


## Generate Annotations

In [9]:
!python /content/PRETSA/calculateAnnotationsEventLog_baseline.py <dictName> #Needs: filePath = dictPath + dataset + "_duration_pretsa_baseline_k" + str(k) + "_t" + str(t) + ".csv" ,Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"
!python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py <dictName> #Needs:dictPath + dataset + "_duration_t" + tString[t] + "_k" + str(k) + "_pretsa.csv" , generates dictPath + "pretsa_statistics_annotations.csv"
!python /content/PRETSA/generateAnnotationOriginalDataset.py <dictName> #generates original_annotations_pretsa.csv (located in <dictName>)

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/calculateAnnotationsEventLog_baseline.py <dictName> #Needs: filePath = dictPath + dataset + "_duration_pretsa_baseline_k" + str(k) + "_t" + str(t) + ".csv" ,Generates: dictPath + "pretsa_baseline_statistics_annotations.csv"'
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/calculateAnnotationsEventLog_pretsa.py <dictName> #Needs:dictPath + dataset + "_duration_t" + tString[t] + "_k" + str(k) + "_pretsa.csv" , generates dictPath + "pretsa_statistics_annotations.csv"'
/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/generateAnnotationOriginalDataset.py <dictName> #generates original_annotations_pretsa.csv (located in <dictName>)'


## Calculate Annotation Errors

 Relative error of the annotations for each activity

In [10]:
!python /content/PRETSA/calculateAnnotationError.py <dictName> #Needs: original_annotations_pretsa.csv, pretsa_baseline_statistics_annotations.csv, pretsa_statistics_annotations.csv

/bin/bash: -c: line 1: syntax error near unexpected token `newline'
/bin/bash: -c: line 1: `python /content/PRETSA/calculateAnnotationError.py <dictName> #Needs: original_annotations_pretsa.csv, pretsa_baseline_statistics_annotations.csv, pretsa_statistics_annotations.csv'
